In [1]:
import findspark 
findspark.init('/home/anycaroliny/spark-3.3.2-bin-hadoop3')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('logistic_regression').getOrCreate()

23/04/04 12:09:05 WARN Utils: Your hostname, anycaroliny-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/04/04 12:09:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 12:09:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.ml.classification import LogisticRegression

In [5]:
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

23/04/04 12:09:12 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
log_reg_model = LogisticRegression()

In [8]:
fitted_log_reg = log_reg_model.fit(data)

23/04/04 12:09:21 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/04 12:09:21 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/04/04 12:09:21 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/04/04 12:09:21 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


In [9]:
log_summary = fitted_log_reg.summary

In [10]:
# log_summary.predictions
# log_summary.predictions.printSchema()
log_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [11]:
lr_train, lr_test = data.randomSplit([0.7, 0.3])

In [12]:
final_model = LogisticRegression()

In [13]:
final_fit = final_model.fit(lr_train)

In [14]:
prediction_and_labels = final_fit.evaluate(lr_test)

In [16]:
prediction_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[121,122,123...|[24.8427523841786...|[0.99999999998374...|       0.0|
|  0.0|(692,[122,123,124...|[17.4972664788862...|[0.99999997482127...|       0.0|
|  0.0|(692,[123,124,125...|[32.3384979048742...|[0.99999999999999...|       0.0|
|  0.0|(692,[123,124,125...|[23.4158600689779...|[0.99999999993229...|       0.0|
|  0.0|(692,[124,125,126...|[24.2776038206396...|[0.99999999997139...|       0.0|
|  0.0|(692,[126,127,128...|[15.1551433315813...|[0.99999973805822...|       0.0|
|  0.0|(692,[126,127,128...|[21.4682264253391...|[0.99999999952524...|       0.0|
|  0.0|(692,[151,152,153...|[35.0851465394841...|[0.99999999999999...|       0.0|
|  0.0|(692,[153,154,155...|[27.7130078629611...|[0.99999999999907...|       0.0|
|  0.0|(692,[153

In [17]:
from pyspark.ml.evaluation import (BinaryClassificationEvaluator,
                                   MulticlassClassificationEvaluator)

In [24]:
evaluation = BinaryClassificationEvaluator()

In [25]:
roc_value = evaluation.evaluate(prediction_and_labels.predictions)

In [27]:
roc_value

1.0